<a href="https://colab.research.google.com/github/secutron/TesTime/blob/main/test_mnist_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np 
import torch.nn.functional as F 

import visdom
vis = visdom.Visdom()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Setting up a new session...


device(type='cuda')

In [ ]:
class fxnnxc(nn.Module):
    def __init__(self):
        super(fxnnxc, self).__init__()
        self.encoder = nn.TransformerEncoderLayer(d_model=28, nhead=4)
        self.linear = nn.Linear(28*28, 10)

    def forward(self, x):
        x = self.encoder(x)
        x = x.view((x.size(0), -1))
        x = self.linear(x)
        return x

In [ ]:
#--------------------------------------------- file read
x_train = np.load("data/mnist_train.npy")
x_test  = np.load("data/mnist_test.npy")
y_train = np.load("data/mnist_train_target.npy")
y_test  = np.load("data/mnist_test_target.npy")
#--------------------------------------------- numpy to tensor
x_train  = torch.from_numpy(x_train).float()       #long으로 하면 loss 계산할 때 에러
x_test   = torch.from_numpy(x_test).float()
y_train  = torch.from_numpy(y_train).long()        #float으로 하면 loss 계산할 때 에러  
y_test   = torch.from_numpy(y_test).long()

#--------------------------------------------- data to dataset
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
test_dataset  = torch.utils.data.TensorDataset(x_test,  y_test)

#--------------------------------------------- dataset to dataloader 
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=256,
                                          shuffle=True,
                                          num_workers=2)
                                    
test_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=256,
                                          shuffle=True,
                                          num_workers=2)                                 
                                    

In [ ]:
model = fxnnxc().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

EPOCH = 1
c = 0
for t in range(EPOCH): # EPOCH
    for i, (sample, target) in enumerate(train_loader): #BATCH
        sample = sample.to(device)
        target = target.to(device)
        y = model(sample)

        loss = criterion(y, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        vis.line(X=[c], Y=[loss.item()], win="loss",name="TransformerEncoder" , 
                    update='append',opts=dict(showlegend=True))
        c+=1

In [ ]:
# Test

correct = 0
total = 0
with torch.no_grad(): 
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(correct/total*100)

87.03999999999999
